# Artificial Neural Network (Sample Test Implementation on hypothetical data) 

In [2]:
# Hypothetical example for Artificial Neural Network
FatScore = c(0.2,0.1,0.2,0.2,0.4,0.3)
SaltScore = c(0.9,0.1,0.4,0.5,0.5,0.8)
Acceptance = c(1,0,0,0,1,1)
FatScore
SaltScore
Acceptance

[1] 0.2 0.1 0.2 0.2 0.4 0.3

[1] 0.9 0.1 0.4 0.5 0.5 0.8

[1] 1 0 0 0 1 1

In [3]:
# Neural Network Structure
# Input layer: two nodes (for two predictors) - nodes 1 and 2
# Hidden Layer: one node - node 3,4,5and 6
# Output Layer: one node - node 6

# THeTA,j and Wij initialization(random number in the range: 0.00 +- 0.05)
bias = matrix(runif(4, -0.05, 0.05), ncol = 1, dimnames = list(c(3,4,5,6), c("THeTA,")))
bias


,"THeTA,"
3,-0.005199481
4,-0.042829957
5,-0.038629767
6,-0.006065988


In [4]:
weightsIH = matrix(runif(6, -0.05, 0.05), ncol = 3, dimnames = list(c(1,2), c(3,4,5)))
weightsHO = matrix(runif(3, -0.05, 0.05), ncol = 1, dimnames = list(c(3,4,5), c(6)))
weightsIH
weightsHO

,3,4,5
1,0.01737694,-0.02597287,-0.0009235633
2,0.01791760,0.03155001,-0.0007025062


,6
3,0.042717210
4,0.001145715
5,0.026686428


In [5]:
# computing output value for first record
# (FastScore = 0.2, Saltscore = 0.9)
output = NULL
k = 1 # first observation
for(i in 1:length(bias)-1){
  x = bias[i,1] + weightsIH[1,i] * FatScore[k] + weightsIH[2,i] * SaltScore[k]
  output[i] = 1/(1 + exp(-x))
}
i = i + 1 # node increment
j = 1

x = bias[i,1] + weightsHO[1,j] * output[1] + weightsHO[2,j] * output[2] + weightsHO[3,j] * output[3]
output[4] = 1/(1 + exp(-x))
output


[1] 0.5036004 0.4950928 0.4901396 0.5072729

In [6]:
# classify first record using cutoff value = 0.5
ifelse(output[4]>0.5, 1,0) #predicted value
Acceptance[1] # actual value


[1] 1

[1] 1

In [7]:
# model for hypothetocal data using library
library(neuralnet)


In [8]:
df = data.frame(FatScore,SaltScore,Acceptance)
str(df)

'data.frame':	6 obs. of  3 variables:
 $ FatScore  : num  0.2 0.1 0.2 0.2 0.4 0.3
 $ SaltScore : num  0.9 0.1 0.4 0.5 0.5 0.8
 $ Acceptance: num  1 0 0 0 1 1


In [13]:
# Neural Network model
# startweights vector: no. of all bias (4) and connection weight values (9)
# linear.outputs = T for predction
# linear.outputs  = F for classification

mod = neuralnet(Acceptance ~ FatScore + SaltScore, df,hidden = c(3), startweights = runif(13, -0.05, 0.05),
                rep = 1,  algorithm = "backprop", learningrate = 0.1, err.fct = "sse", linear.output = F)
mod$result.matrix[1:3,1]
summary(mod)

error reached.threshold             steps 
   0.749999195921    0.001861634949    1.000000000000

                    Length Class      Mode    
call                10     -none-     call    
response             6     -none-     numeric 
covariate           12     -none-     numeric 
model.list           2     -none-     list    
err.fct              1     -none-     function
act.fct              1     -none-     function
linear.output        1     -none-     logical 
data                 3     data.frame list    
net.result           1     -none-     list    
weights              1     -none-     list    
startweights         1     -none-     list    
generalized.weights  1     -none-     list    
result.matrix       16     -none-     numeric 

In [14]:
# inter layer connection weights
# input layer to hidden layer connections
dimnames(mod$weights[[1]][[1]]) = list(c("bias", "node1:fat", "node2:salt"), c("node3","node4","node5"))

# hidden layer to output layer connections
dimnames(mod$weights[[1]][[2]]) = list(c("bias", "node3", "node4", "node5"), c("node6:accept"))

In [15]:
# classify training records
modtrainc = ifelse(mod$net.result[[1]][,1]>0.5, 1, 0)
modtrainc = unname(modtrainc)

data.frame("predicted class" = modtrainc, "Actual class" = df$Acceptance, 
           "predicted value" = mod$net.result[[1]][,1], "fat" = df$FatScore, "salt" = df$SaltScore)

predicted.class,Actual.class,predicted.value,fat,salt
0,1,0.4995052174,0.2,0.9
0,0,0.4995109060,0.1,0.1
0,0,0.4995116664,0.2,0.4
0,0,0.4995103767,0.2,0.5
0,1,0.4995196353,0.4,0.5
0,1,0.4995111365,0.3,0.8


In [16]:
# classification matrix
table("Actual class" = df$Acceptance, "predicted class" = factor(modtrainc, levels = c("0","1")))

            predicted class
Actual class 0 1
           0 3 0
           1 3 0

In [17]:
# classification accuracy
mean(modtrainc == df$Acceptance)
# misclassification accuracy
mean(modtrainc != df$Acceptance)

[1] 0.5

[1] 0.5